In [ ]:
%env MONGO_PW changeme
import os
import pymongo
mongoUri = 'mongodb://{id}:{pw}@{endpoint}/{db}?authSource={db}'
client = pymongo.MongoClient(
    mongoUri.format(
        id="2020sw",
        pw=os.environ['MONGO_PW'],
        endpoint="doctor.iptime.org",
        db="2020sw"
    ))
db = client.get_database()

print("Available collections:")
for c in db.list_collection_names():
    print("\t", c, db.get_collection(c).estimated_document_count())

In [ ]:
# json 파일을 그대로 DB에 넣어버리는 코드
col = client.get_database().get_collection("articles")
col.create_index([('date', pymongo.ASCENDING)])

from datetime import datetime

for fname in os.listdir('data'):
    with open(os.path.join('data', fname)) as f:
        data = json.load(f)
        for o in data:
            o['date'] = datetime.strptime(o['date'], "%Y.%m.%d")
        col.insert_many(data)
    print(fname)

In [ ]:
# articles에 들어있는 기사에서 명사(keyword) 추출하는 코드

# %%
print("Init Collection")
col = db.get_collection("articles")
col.update_many({"status": { "$exists":False }}, {"$set": {"status": 0}}) # 새로 추가된 기사
col.update_many({"status": 1}, {"$set": {"status": 0}}) # 파싱 안된 기사
col.create_index([("status", pymongo.DESCENDING)])
# %%
print("Init Kkma")
from konlpy.tag import Kkma
km = Kkma()
# %%
col = db.get_collection("articles")
while True:
    doc = col.find_one_and_update({"status" : 0}, {"$set": {"status": 1}})
    if doc == None:
        break
    print(doc['link'], doc['title'])
    keywords = {}
    for noun in km.nouns(doc['body']):
        if len(noun) < 2:
            continue
        if noun not in keywords:
            keywords[noun] = 0
        keywords[noun] += 1
    col.find_one_and_update({"_id": doc["_id"]}, {"$set": {"status": 2, "keywords": keywords}})
    print(doc['link'], doc['title'], len(keywords))
print("Finished")

In [ ]:
# status: 2 에서 status: 3 포맷으로 바꾸는 코드
col = db.get_collection("articles")
while True:
    doc = col.find_one({"status": 2})
    if doc == None:
        break
    keywords = []
    for k in doc["keywords"].keys():
        keywords.append({
            "name": k,
            "value": doc["keywords"][k]
        })
    col.find_one_and_update({"_id": doc['_id']}, {"$set": { "keywords": keywords, "status": 3}})
    print(doc['link'], doc['title'], len(keywords))
print("Finished")

In [ ]:
# pagerank 작업할 DB placeholder 만드는 코드
from datetime import datetime, timedelta
from IPython.display import clear_output

articles = db.get_collection("articles")
ranked = db.get_collection("ranked")
ranked.create_index([("status", pymongo.DESCENDING)])
ranked.create_index([("date", pymongo.DESCENDING)], unique=True)
for date in articles.find({"status": 3}).distinct("date"):
    clear_output(wait=True)
    print(date)
    ranked.insert_one({
        "date": date,
        "status": 0,
        "keywords": []
    })